# Libraries

In [18]:
import pandas as pd

# Step 1: Data

In [69]:
airbnb_data=pd.read_csv("Airbnb_Texas_Rentals.csv")

# Step 2: Create documents

In [21]:
airbnb_data.shape

(18259, 9)

In [20]:
def create_tsv_documents():
    """
    Method that creates different .tsv files for each record in the airbnb_data 
    """
    
    #rename unnamed column as an ID column
    airbnb_data.rename(columns={'Unnamed: 0':'ID'},inplace=True)
    #set ID column as index
    airbnb_data=airbnb_data.set_index('ID')
    #for each index make a dataframe of airbnb_data and store it into new tsv file
    for i in airbnb_data.index:
        pd.DataFrame(airbnb_data.loc[i]).transpose().to_csv('data/doc_'+str(i)+'.tsv',sep='\t')

In [ ]:
#method is run only once at the beginning to make separate .tsv files
create_tsv_documents()

for i in airbnb_data.index:
    x=pd.read_csv('data/doc_'+str(i)+'.tsv',sep='\t',encoding='ISO-8859-1',index_col='ID',de)

In [60]:
i=5
x=pd.read_csv('data/doc_'+str(i)+'.tsv',sep='\t',encoding='ISO-8859-1')

In [61]:
x

,Unnamed: 0,average_rate_per_night,bedrooms_count,city,date_of_listing,description,latitude,longitude,title,url
0,5,$75,2,Fort Worth,February 2017,Welcome to our original 1920's home. We recent...,32.747097,-97.286434,The Porch,https://www.airbnb.com/rooms/17325114?location...
